<a href="https://colab.research.google.com/github/War-Thib/ForYouLoic/blob/CleanPharmaCycle/OM_Assignment_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. Getting the data


In [ ]:
import numpy as np 
import pandas as pd

dataset = "assignment_4_dataset.csv"

saintx_data = pd.read_csv(dataset)

saintx_data

# 1. Naive Forecast, MA3, MA5 and exponential smoothing

In [ ]:
beers = ['Donker', 'Red', 'Tripel']

def moving_average(series, order): 
    result = np.full(len(series) + 1, np.NaN)
    for idx in range(order - 1, len(series) + 1 ):
        result[idx] = np.mean(series[idx-order:idx])
    return result

def exponential_smoothing(series, alpha = 0.5):
    result = np.full(len(series) + 1, np.NaN)
    result[1] = series[0]
    for idx in range(2, len(series) + 1):
        result[idx] = alpha * series[idx - 1] + \
        (1 - alpha) * result[idx - 1]
    return result   

def double_exponential_smoothing(series, alpha, beta):
  
  forecast = np.full(len(series) + 1, np.NaN)
  a = np.full(len(series)+1, np.NaN)  # a et b sont des étapes intermédiaires nécessaires au calcul des prévisions
  b = np.full(len(series)+1, np.NaN)

  a[0] = series[0]
  a[1] = series[1]
  b[1] = series[1] - series[0]

  
  for idx in range(2, len(series)):
      forecast[idx] = a[idx - 1] + b[idx - 1]
      a[idx] = alpha * series[idx] + (1 - alpha) * (a[idx - 1] + b[idx - 1])
      b[idx] = beta * (a[idx] - a[idx - 1]) + (1 - beta) * b[idx - 1]
  
  forecast[-1] = a[-2] + b[-2]
      
  return forecast     


for b in beers:
  #naive forecast
  saintx_data["Naive_forecast_"+b] = saintx_data[b +" Drunk"].shift(1)
  # moving average 3-5
  saintx_data[b+"_MA3_forecast"] = moving_average(saintx_data[b + " Drunk"][:-1], 3)
  saintx_data[b+"_MA5_forecast"] = moving_average(saintx_data[b + " Drunk"][:-1], 5)
  # simple exponential smoothing
  saintx_data["SES_"+b] = exponential_smoothing(saintx_data[b +' Drunk'][:-1], 0.2)
  # exponential smoothing with trend
  saintx_data["DES_" + b] = double_exponential_smoothing(saintx_data[b + " Drunk"][:-1], 0.5, 0.5)


saintx_data = saintx_data.drop([0,1,2,3,4]) 

saintx_data = saintx_data.reset_index(drop=True)


# saintx_data


# 2. The best error measure (MSE) 

## Why ?
Accoring to us, the best error measure in this case is the MSE. In this forecasting case, only the really big errors_donker will create dramatic situations (bankrupcy and anger). And the MSE, is giving more weight in the balance to important errors_donker. That's the reason why we'll choose the forecast model which scores best whith as error measure the MSE.

## The best forecasting model according to the MSE ?

The best forecasting model according to the MSE, is the Moving average order 5. With an error of 21390.49



In [ ]:
# let's calculate the MSE error for each model for each beer, pour le MSE j'ai besoin de l'erreur et de l'erreur au carré
beers = ['Donker', 'Red', 'Tripel']
squared = lambda x: x**2

for b in beers:
  # error + squared error for naive forecast
  saintx_data['err_naive_'+b] = saintx_data[b+' Drunk'] - saintx_data['Naive_forecast_'+b]
  saintx_data['squarred_err_naive_'+b] =  list(map(squared, saintx_data['err_naive_'+b]))

  # error + squarred error for MA3-MA5
  saintx_data['err_MA3_'+b] = saintx_data[b+' Drunk'] - saintx_data[b+'_MA3_forecast']
  saintx_data['squarred_err_MA3_'+b] =  list(map(squared, saintx_data['err_MA3_'+b]))
  saintx_data['err_MA5_'+b] = saintx_data[b+' Drunk'] - saintx_data[b+'_MA5_forecast']
  saintx_data['squarred_err_MA5_'+b] =  list(map(squared, saintx_data['err_MA5_'+b]))

  # error + squarred error for simple exponential smoothing
  saintx_data['err_SES_'+b] = saintx_data[b+' Drunk'] - saintx_data['SES_'+b]
  saintx_data['squarred_err_SES_'+b] =  list(map(squared, saintx_data['err_SES_'+b]))

  # error + squarred error for double exponential smoothing
  saintx_data['err_DES_'+b] = saintx_data[b+' Drunk'] -saintx_data['DES_'+b]
  saintx_data['squarred_err_DES_'+b] =  list(map(squared, saintx_data['err_DES_'+b]))

#il y a blindé de colones c'est abusé
saintx_data['constante_pour_MSE'] = saintx_data['Donker Drunk']/saintx_data['Donker Drunk'] #va me permettre de générer une colone de constante afin que je fasse mon groupby là dessus, c'est très laid mais ça va marcher

# on en vient aux choses sérieuse je dois ici calculer le MSE 

mse = saintx_data.groupby("constante_pour_MSE").aggregate([np.sum, len])

for b in beers:
  mse['mse_naive_'+b] = mse['squarred_err_naive_'+b]['sum']/mse['squarred_err_naive_'+b]['len']
  mse['mse_MA3_'+b] = mse['squarred_err_MA3_'+b]['sum']/mse['squarred_err_MA3_'+b]['len']
  mse['mse_MA5_'+b] = mse['squarred_err_MA5_'+b]['sum']/mse['squarred_err_MA5_'+b]['len']
  mse['mse_SES_'+b] = mse['squarred_err_SES_'+b]['sum']/mse['squarred_err_SES_'+b]['len']
  mse['mse_DES_'+b] = mse['squarred_err_DES_'+b]['sum']/mse['squarred_err_DES_'+b]['len']

# afin de choisir le meilleur modèle => faire la somme des mse 

mse['sum_mse_naive'] = mse['mse_naive_Donker'] + mse['mse_naive_Red'] + mse['mse_naive_Tripel']
mse['sum_mse_MA3'] = mse['mse_MA3_Donker'] + mse['mse_MA3_Red'] + mse['mse_MA3_Tripel']
mse['sum_mse_MA5'] = mse['mse_MA5_Donker'] + mse['mse_MA5_Red'] + mse['mse_MA5_Tripel']
mse['sum_mse_SES'] = mse['mse_SES_Donker'] + mse['mse_SES_Red'] + mse['mse_SES_Tripel']
mse['sum_mse_DES'] = mse['mse_DES_Donker'] + mse['mse_DES_Red'] + mse['mse_DES_Tripel']


first_mse_model = mse[['sum_mse_naive','sum_mse_MA3', 'sum_mse_MA5','sum_mse_SES','sum_mse_DES']]
first_mse_model



,sum_mse_naive,sum_mse_MA3,sum_mse_MA5,sum_mse_SES,sum_mse_DES
,,,,,
constante_pour_MSE,,,,,
1.0,29623.088889,22187.217284,21390.493333,22907.680336,28535.451443


#3. Some changes in the modelling of our forecasting model.

The student association is giving us the information that if the forecast is too short in kegs, => all the sales losts are valuated at 2€.
In the other hand, if the forecast is too large => all the beers remaining in the kegs are valuated at 1€.

We need to implement thoses informations in the computation of our errors_donker. Just giving twice the weight to the negative errors_donker.


En gros pour moi, il faut garder le MSE comme calcul d'erreur mais il faut ajouter 2fois plus de poids aux erreur négatives


Ce que je propose c'est de faire c'est d'ajouter ces changements à notre base de donnée et doublant la valeur de l'erreur pour les erreurs négatives. En faisant cela on obtiendra des autres MSE et j'espère vraiment que ça sera un des trucs exponential qui gagnera 


Celui qui gagne c'est le forecast en MA3 avec une mse de 60151.93

In [ ]:
# ici cette partie de code génère la colonne qui contient la nouvelle version de l'erreur qui est 2 fois plus importante si elle est négative

for b in beers:
  saintx_data['err_model_naive_'+b] = np.where(saintx_data['err_naive_'+b]>=0, saintx_data['err_naive_'+b]*2, saintx_data['err_naive_'+b]) 
  saintx_data['err_model_MA3_'+b] = np.where(saintx_data['err_MA3_'+b]>=0, saintx_data['err_MA3_'+b]*2, saintx_data['err_MA3_'+b])
  saintx_data['err_model_MA5_'+b] = np.where(saintx_data['err_MA5_'+b]>=0, saintx_data['err_MA5_'+b]*2, saintx_data['err_MA5_'+b])
  saintx_data['err_model_SES_'+b] = np.where(saintx_data['err_SES_'+b]>=0, saintx_data['err_SES_'+b]*2, saintx_data['err_SES_'+b])
  saintx_data['err_model_DES_'+b] = np.where(saintx_data['err_DES_'+b]>=0, saintx_data['err_DES_'+b]*2, saintx_data['err_DES_'+b])


  
  #now let's create the squarred err of our new model_error
  saintx_data['squarred_err_model_naive_'+b] =  list(map(squared, saintx_data['err_model_naive_'+b]))
  saintx_data['squarred_err_model_MA3_'+b] =  list(map(squared, saintx_data['err_model_MA3_'+b]))
  saintx_data['squarred_err_model_MA5_'+b] =  list(map(squared, saintx_data['err_model_MA5_'+b]))
  saintx_data['squarred_err_model_SES_'+b] =  list(map(squared, saintx_data['err_model_SES_'+b]))
  saintx_data['squarred_err_model_DES_'+b] =  list(map(squared, saintx_data['err_model_DES_'+b]))


mse_model = saintx_data.groupby('constante_pour_MSE').aggregate([np.sum, len])
mse_model = pd.DataFrame(mse_model)

for b in beers:
  mse_model['mse_model_naive_'+b] = mse_model['squarred_err_model_naive_'+b]['sum']/mse_model['squarred_err_model_naive_'+b]['len']
  mse_model['mse_model_MA3_'+b] = mse_model['squarred_err_model_MA3_'+b]['sum']/mse_model['squarred_err_model_MA3_'+b]['len']
  mse_model['mse_model_MA5_'+b] = mse_model['squarred_err_model_MA5_'+b]['sum']/mse_model['squarred_err_model_MA5_'+b]['len']
  mse_model['mse_model_SES_'+b] = mse_model['squarred_err_model_SES_'+b]['sum']/mse_model['squarred_err_model_SES_'+b]['len']
  mse_model['mse_model_DES_'+b] = mse_model['squarred_err_model_DES_'+b]['sum']/mse_model['squarred_err_model_DES_'+b]['len']


#afin de choisir le meilleur modèle on fait la somme des 3
mse_model['sum_mse_model_naive'] = mse_model['mse_model_naive_Donker'] + mse_model['mse_model_naive_Red'] + mse_model['mse_model_naive_Tripel']
mse_model['sum_mse_model_MA3'] = mse_model['mse_model_MA3_Donker'] + mse_model['mse_model_MA3_Red'] + mse_model['mse_model_MA3_Tripel']
mse_model['sum_mse_model_MA5'] = mse_model['mse_model_MA5_Donker'] + mse_model['mse_model_MA5_Red'] + mse_model['mse_model_MA5_Tripel']
mse_model['sum_mse_model_SES'] = mse_model['mse_model_SES_Donker'] + mse_model['mse_model_SES_Red'] + mse_model['mse_model_SES_Tripel']
mse_model['sum_mse_model_DES'] = mse_model['mse_model_DES_Donker'] + mse_model['mse_model_DES_Red'] + mse_model['mse_model_DES_Tripel']

final_mse_model = mse_model[['sum_mse_model_naive','sum_mse_model_MA3','sum_mse_model_MA5','sum_mse_model_SES', 'sum_mse_model_DES' ]]
final_mse_model

,sum_mse_model_naive,sum_mse_model_MA3,sum_mse_model_MA5,sum_mse_model_SES,sum_mse_model_DES
,,,,,
constante_pour_MSE,,,,,
1.0,74351.422222,60151.935802,61686.693333,72608.774531,67529.231449


#4. Which forecasting model ? (with the changes made in the part 3) With which parameters ?

Even with the new error calculation, it still is the MA5 that outperform the other forecasting models.
let's now see if we can optimise the parameter of the moving average.

ici le problème c'est que je n'ai pas prit en compte les erreurs qui sont changées 



## 4.1 Finding the best moving average order for the Donker Drunk


The order that optimize the forecast is **4**

In [ ]:
mas_donker = []

saintx_data2 = pd.read_csv('assignment_4_dataset.csv')

for order in range(1, 6):
    forecast_donker = pd.DataFrame({"Year" : saintx_data2["Year"],
                            "Order" : order,
                            "Donker Drunk" : saintx_data2["Donker Drunk"]})
    forecast_donker["Forecast Donker"] = moving_average(saintx_data2["Donker Drunk"], order)[:-1] #ici je dois comprend pourquoi est-ce que je ne dois pas mettre de [:-1]
    mas_donker.append(forecast_donker)

mas_donker = pd.concat(mas_donker, ignore_index = True)


In [ ]:
errors_donker = mas_donker.copy()

errors_donker = errors_donker.loc[errors_donker.Year >= 1975]
errors_donker["Error"] = errors_donker["Donker Drunk"] - errors_donker['Forecast Donker']

errors_donker["Model Error"] = np.where(errors_donker["Error"] >=0, errors_donker["Error"]*2, errors_donker["Error"])
errors_donker["Model Squarred Error"] = list(map(squared, errors_donker["Model Error"]))

mse_donker = errors_donker.groupby("Order").aggregate([np.sum, len]).reset_index()

mse_donker = pd.DataFrame({"Order" : mse_donker["Order"],
                    "MSE" : mse_donker["Model Squarred Error"]["sum"] / mse_donker["Model Squarred Error"]["len"]})

mse_donker

,Order,MSE
0,1,36536.533333
1,2,30320.083333
2,3,29194.271605
3,4,28161.644444
4,5,30180.723556


## 4.2 Finding the best moving average order for the Red Drunk

The order that optimize the forecast is **2**

In [ ]:
mas_red = []

for order in range(1, 6): # ici voir si j'investigue vers plus loin que ordre 10 ou pas => car le truc qui optimise le plus le modèle c'est ordre 13
    forecast_red = pd.DataFrame({"Year" : saintx_data2["Year"],
                            "Order" : order,
                            "Red Drunk" : saintx_data2["Red Drunk"]})
    forecast_red["Forecast Red"] = moving_average(saintx_data2["Red Drunk"], order)[:-1] #ici je dois comprend pourquoi est-ce que je ne dois pas mettre de [:-1]
    mas_red.append(forecast_red)

mas_red = pd.concat(mas_red, ignore_index = True)

In [ ]:
errors_red = mas_red.copy()

errors_red = errors_red.loc[ errors_red.Year >= 1975]

errors_red["Error"] = errors_red["Red Drunk"] - errors_red['Forecast Red']

errors_red["Model Error"] = np.where(errors_red["Error"] >=0, errors_red["Error"]*2, errors_red["Error"])
errors_red["Model Squarred Error"] = list(map(squared, errors_red["Model Error"]))


mse_red = errors_red.groupby("Order").aggregate([np.sum, len]).reset_index()

mse_red = pd.DataFrame({"Order" : mse_red["Order"],
                    "MSE" : mse_red["Model Squarred Error"]["sum"] / mse_red["Model Squarred Error"]["len"]})

mse_red

,Order,MSE
0,1,24208.666667
1,2,19414.450000
2,3,19617.864198
3,4,20748.759722
4,5,21240.401778


## 4.3 Finding the best moving average order for the tripel red

The order that optimize the forecast is **5**

In [ ]:
mas_tripel = []

for order in range(1, 6):
    forecast_tripel = pd.DataFrame({"Year" : saintx_data2["Year"],
                            "Order" : order,
                            "Tripel Drunk" : saintx_data2["Tripel Drunk"]})
    forecast_tripel["Forecast Tripel"] = moving_average(saintx_data2["Tripel Drunk"], order)[:-1] #ici je dois comprend pourquoi est-ce que je ne dois pas mettre de [:-1]
    mas_tripel.append(forecast_tripel)

mas_tripel = pd.concat(mas_tripel, ignore_index = True)

In [ ]:
errors_tripel = mas_tripel.copy()

errors_tripel = errors_tripel.loc[errors_tripel.Year >= 1975]

errors_tripel["Error"] = errors_tripel["Tripel Drunk"] - errors_tripel['Forecast Tripel']
errors_tripel["Model Error"] = np.where(errors_tripel["Error"] >=0, errors_tripel["Error"]*2, errors_tripel["Error"])
errors_tripel["Model Squarred Error"] = list(map(squared, errors_tripel["Model Error"]))

mse_tripel = errors_tripel.groupby("Order").aggregate([np.sum, len]).reset_index()

mse_tripel = pd.DataFrame({"Order" : mse_tripel["Order"],
                    "MSE" : mse_tripel["Model Squarred Error"]["sum"] / mse_tripel["Model Squarred Error"]["len"]})

mse_tripel

,Order,MSE
0,1,13606.222222
1,2,13003.855556
2,3,11339.800000
3,4,10342.647222
4,5,10265.568000


#5 What is your best estimate of the number of kegs to buy in 2020? Why (explain how you came to this recommendation)?

Our results from the question 3:

*   Best forecasting method for Donker Drunk = MA4
*   Best forecasting method for Red Drunk = MA2
*   Best forecasting method for Tripel Drunk = MA5

-----------------------------------------------------------------

It's also time to think in terms of kegs and not of beers ( 1 kegs = 80 beers) The only remaining think to calculate is for the optimization of kegs => knowing that one beer remaining in the kegs = 1€ loss but 1 sale loss = 2€. This think is important to calculate the exact number of kegs we need.




In [ ]:
int(moving_average(saintx_data2["Donker Drunk"], 4)[-1])

884

In [ ]:
int(moving_average(saintx_data2["Red Drunk"], 2)[-1])

300

In [ ]:
int(moving_average(saintx_data2["Tripel Drunk"], 5)[-1])

88